# Information gain(同質)(不檢測)

In [29]:
####import####
import math 
from collections import Counter
import pandas as pd
import random  
from pandas.core.frame import DataFrame
import numpy as np 
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import ensemble,metrics

####讀檔####
df = pd.read_csv('vehicle.csv')  
x=df.iloc[:,:-1]
y=df.iloc[:,-1]





#離散化(kfold外)
i_x=pd.DataFrame(columns=x.columns)
for i in range(x.shape[1]):
    temp_=x.iloc[:,i]
    xd= pd.cut(temp_, 5,labels=range(1,6))
    i_x.iloc[:,i]=xd


####kfold####
kf=KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in kf.split(x):
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42) 

    ####參數####
    ff=0 #feature從0開始跑 
    m=5#基本模型數量
    feature_number=2 #特徵數
    random_predict=[]#預測x_train
    d_random_predict=[]#預測d_x
    
    
    ####測試多樣性的樣本####
    #d_df=df.sample(frac=0.3, replace=False, random_state=1)
    #d_x=d_df.iloc[:,:-1]
    #d_y=d_df.iloc[:,-1]
    d_x=x_train
    d_y=y_train
    
    ####計算entropy####
    #x
    i_x_train=i_x.iloc[train_index] #676*18

    #y
    y_count=y_train.value_counts(normalize=True).sort_index()#數量/總數 並按index排序
    entropy=0
    for e in range(len(y_count)):
        entropy+=-y_count[e]*(math.log(y_count[e],2))

    #類別也變1,2,3...
    i_y_train=y_train 
    for e in range(len(y_count)):
        i_y_train=i_y_train.replace(y_count.index[e], e) 

    #每個特徵的排
    e_rank=[]# 每個特徵的information gain
    for e in range(i_x_train.shape[1]):#幾個特徵i_x_train.shape[1]
        temp_feature=[]
        temp_feature.append(i_y_train)
        temp_feature.append(i_x_train.iloc[:,e])
        temp_feature=DataFrame(temp_feature)

        feature_count=i_x_train.iloc[:,e].value_counts().sort_index() #feature有幾個特徵值
        e=0 #特徵的information gain
        #print(feature_count.iloc[0])

        for f in range(1,len(feature_count)+1):#1~5
            temp_count_class=[]  #------------------------------
            locals()['entropy_'+str(f)]=0 #------------------------------
            for h in range(i_x_train.shape[0]):#677
                if temp_feature.iloc[1,h]==f: #存所有特徵質為0的y
                    temp_count_class.append(temp_feature.iloc[0,h])
            
            for h in range(len(y_count)):#0~3
                if temp_count_class.count(h)!=0:
                    locals()['entropy_'+str(f)]+=-(temp_count_class.count(h)/feature_count.iloc[0])*math.log(temp_count_class.count(h)/feature_count.iloc[0],2)
        for f in range(1,len(feature_count)+1):#0~4
            e+=feature_count.iloc[f-1]/i_x_train.shape[0]*locals()['entropy_'+str(f)]
        e_rank.append(e)


    rank=sorted(range(len(e_rank)), key=lambda k: e_rank[k]) #排序後的index 
    print(rank)


    ####Bagging####
    '''
    random_1 = ensemble.BaggingClassifier(DecisionTreeClassifier(),n_estimators = 1)
    random_fit_1 = random_1.fit(x_train, y_train)        
    y_predict_1 = random_1.predict(x_test) 
    accuracy_1 = metrics.accuracy_score(y_test,y_predict_1)
    print("random_1 : ",accuracy_1)
    '''
    feature_data=[]
    for i in range(1):#(1,m+1)
        for feature_point in range(ff,ff+feature_number):
            feature_data.append(x_train.iloc[:,rank[feature_point]])
        ff+=1
        feature_data=DataFrame(feature_data).T
        locals()['x_train'+str(i)]=feature_data
        locals()['y_train'+str(i)]=y_train[locals()['x_train'+str(i)].index]
        locals()['random_'+str(i)]=DecisionTreeClassifier()
        locals()['random_'+str(i)].fit(locals()['x_train'+str(i)], locals()['y_train'+str(i)])
        locals()['y_predict_'+str(i)]=locals()['random_'+str(i)].predict(x_test)
        random_predict.append(locals()['y_predict_'+str(i)])#存每個Base預測x_test的結果
        #(每個基本模型的準確度)locals()['accuracy_'+str(i)]= metrics.accuracy_score(y_test,locals()['y_predict_'+str(i)])
        #(每個基本模型的準確度)print(i," : " ,locals()['accuracy_'+str(i)])



    ####acc####
    #(樣本數) random_predict.shape[1] 
    #(模型數) random_predict.shape[0]
    temp=[]
    acc_count=0
    random_predict=np.array(random_predict)#list轉array
    for i in range(random_predict.shape[1]):
        temp=[]
        for j in range(random_predict.shape[0]):
            temp.append(random_predict[j][i])
        vote= max(temp,key=temp.count)
        if vote==y_test.iloc[i]:
            acc_count+=1

    print("acc : ",acc_count/random_predict.shape[1] )


    ####Diversity#### 
    d_random_predict.append(d_y) #第0列放實際值


    '''    
    d_random_predict.append(bag_1.predict(d_x))
    '''
    for i in range(1,11): #第1~10列放Model預測值
        d_random_predict.append(locals()['random_'+str(i)].predict(d_x))
        
    d_random_predict=np.array(d_random_predict)#list轉array



    #(樣本數) d_random_predict.shape[1] 
    #(模型數) d_random_predict.shape[0]


    #轉為O或1
    for i in range(d_random_predict.shape[1]):
        for j in range(1,d_random_predict.shape[0]):
            if d_random_predict[j][i]==d_random_predict[0][i]:
                d_random_predict[j][i]=1 #預測正確
            else:
                d_random_predict[j][i]=0 #預測錯誤



    #pi參數設定          
    for i in range(d_random_predict.shape[0]):
        locals()['p'+str(i)]=0
    cfd=0

    #計算pi
    for i in range(d_random_predict.shape[1]):
        count=0
        for j in range(1,d_random_predict.shape[0]):
            if d_random_predict[j][i]==0:
                count+=1
        '''
        if count==0:
            p0+=1
        '''
        for j in range(d_random_predict.shape[0]+1):
            if count==j:
                locals()['p'+str(j)]+=1


    #數量/總數        
    for i in range(d_random_predict.shape[0]):
        locals()['p'+str(i)]=locals()['p'+str(i)]/d_random_predict.shape[1]

    #計算CFD
    x1=0
    x2=0

    if p0==1:
        cfd=0
    else:
        x1=1/(1-p0)
        for i in range(1,d_random_predict.shape[0]):
            x2=(d_random_predict.shape[0]-i)/(d_random_predict.shape[0]-1)*locals()['p'+str(i)]+x2
        cfd=x1*x2   

    print("cfd : ",cfd)
    
    
#print(x_train1.shape)

[2, 0, 8, 6, 11, 7, 10, 14, 13, 15, 4, 17, 16, 9, 3, 12, 1, 5]


ValueError: Number of features of the model must match the input. Model n_features is 2 and input n_features is 18 

In [50]:

#計算entropy

#x
i_x_train=.iloc[train_index]

#y
y_count=y_train.value_counts(normalize=True).sort_index()#數量/總數 並按index排序
entropy=0
for e in range(len(y_count)):
    entropy+=-y_count[e]*(math.log(y_count[e],2))
    
#類別也變1,2,3...
i_y_train=y_train 
for e in range(len(y_count)):
    i_y_train=i_y_train.replace(y_count.index[e], e) 


#每個特徵的排
e_rank=[]# 每個特徵的information gain
for e in range(i_x_train.shape[1]):#幾個特徵
    temp_feature=[]
    temp_feature.append(i_y_train)
    temp_feature.append(i_x_train.iloc[:,e])
    feature_count=i_x_train.iloc[:,e].value_counts().sort_index() #feature有幾個特徵值
    e=0 #特徵的information gain
    #print(feature_count.iloc[0])
    for f in range(len(feature_count)):#0~4
        temp_count_class=[]  #------------------------------
        locals()['entropy_'+str(f)]=0 #------------------------------
        for h in range(i_x_train.shape[0]):#677
            if temp_feature[1][h]==f:
                temp_count_class.append(temp_feature[0][h])
        for h in range(len(y_count)):#0~3
            if temp_count_class.count(h)!=0:
                locals()['entropy_'+str(f)]+=-(temp_count_class.count(h)/feature_count.iloc[0])*math.log(temp_count_class.count(h)/feature_count.iloc[0],2)
    for f in range(len(feature_count)):#0~4
        e+=feature_count.iloc[f]/i_x_train.shape[0]*locals()['entropy_'+str(f)]
    e_rank.append(e)
    
rank=sorted(range(len(e_rank)), key=lambda k: e_rank[k]) #排序後的index 
print(rank)

[2, 0, 8, 6, 11, 7, 15, 14, 10, 13, 4, 9, 1, 17, 3, 16, 12, 5]


In [9]:
####import####
import math 
from collections import Counter
import pandas as pd
import random  
from pandas.core.frame import DataFrame
import numpy as np 
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import ensemble,metrics

####讀檔####
df = pd.read_csv('vehicle.csv')  
x=df.iloc[:,:-1]
y=df.iloc[:,-1]


####測試多樣性的樣本####
d_df=df.sample(frac=0.3, replace=False, random_state=1)
d_x=d_df.iloc[:,:-1]
d_y=d_df.iloc[:,-1]


#離散化(kfold外)
i_x=pd.DataFrame(columns=x_train.columns)
for i in range(x.shape[1]):
    temp_=x.iloc[:,i]
    xd= pd.cut(temp_, 5,labels=range(1,6))
    i_x.iloc[:,i]=xd

print(i_x)
####kfold####
kf=KFold(n_splits=5, random_state=None, shuffle=False)
for train_index, test_index in kf.split(x):
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42) 

    ####參數####
    m=10#基本模型數量
    random_predict=[]#預測x_train
    d_random_predict=[]#預測d_x

    
    ####計算entropy####
    #x
    i_x_train=i_x.iloc[train_index] #676*18

    #y
    y_count=y_train.value_counts(normalize=True).sort_index()#數量/總數 並按index排序
    entropy=0
    for e in range(len(y_count)):
        entropy+=-y_count[e]*(math.log(y_count[e],2))

    #類別也變1,2,3...
    i_y_train=y_train 
    for e in range(len(y_count)):
        i_y_train=i_y_train.replace(y_count.index[e], e) 

    #每個特徵的排
    e_rank=[]# 每個特徵的information gain
    for e in range(i_x_train.shape[1]):#幾個特徵i_x_train.shape[1]
        temp_feature=[]
        temp_feature.append(i_y_train)
        temp_feature.append(i_x_train.iloc[:,e])
        temp_feature=DataFrame(temp_feature)

        feature_count=i_x_train.iloc[:,e].value_counts().sort_index() #feature有幾個特徵值
        e=0 #特徵的information gain
        #print(feature_count.iloc[0])

        for f in range(1,len(feature_count)+1):#1~5
            temp_count_class=[]  #------------------------------
            locals()['entropy_'+str(f)]=0 #------------------------------
            for h in range(i_x_train.shape[0]):#677
                if temp_feature.iloc[1,h]==f: #存所有特徵質為1的y
                    temp_count_class.append(temp_feature.iloc[0,h])
            
            for h in range(len(y_count)):#0~3
                if temp_count_class.count(h)!=0:
                    locals()['entropy_'+str(f)]+=-(temp_count_class.count(h)/feature_count.iloc[0])*math.log(temp_count_class.count(h)/feature_count.iloc[0],2)
        for f in range(1,len(feature_count)+1):#0~4
            e+=feature_count.iloc[f-1]/i_x_train.shape[0]*locals()['entropy_'+str(f)]
        e_rank.append(e)


    rank=sorted(range(len(e_rank)), key=lambda k: e_rank[k]) #排序後的index 
    print(rank)


    COMPACTNESS CIRCULARITY DISTANCE CIRCULARITY RADIUS RATIO  \
0             3           3                    3            2   
1             2           2                    4            1   
2             4           4                    5            3   
3             3           2                    3            2   
4             2           3                    3            3   
..          ...         ...                  ...          ...   
841           3           2                    4            2   
842           2           3                    4            2   
843           4           5                    5            3   
844           2           1                    3            1   
845           2           1                    2            1   

    PR.AXIS ASPECT RATIO MAX.LENGTH ASPECT RATIO SCATTER RATIO ELONGATEDNESS  \
0                      2                       1             2             3   
1                      1                       1           

In [ ]:
   locals()['entropy_'+str(f)]=0
        for g in range(len(y_count)):#0~3
            count=0
           
                if temp_feature[0][h]==g and :
                    count+=1
        locals()['entropy_'+str(f)]+=-(count/feature_count[f])*math.log((count/feature_count[f]),2)


In [ ]:
from collections import Counter
ll=Counter(x.iloc[:,0])
print(ll)
ll.values()


In [ ]:
df.iloc[:,0].value_counts

# 驗證

In [ ]:

####參數####
m=10#基本模型數量
bag_predict=[]#預測x_train
d_bag_predict=[]#預測d_x

b=ensemble.BaggingClassifier(DecisionTreeClassifier(),n_estimators = 10)
b.fit(x_train, y_train)
estimators_samples = b.estimators_samples_
estimators_samples=np.array(estimators_samples)
accuracy=b.score(x_test,y_test)
print(accuracy)

#y_predict= b.predict(x_test) 
#accuracy = metrics.accuracy_score(y_test,y_predict)
#print("Bagging: ",accuracy)

#處理子訓練資料
for i in range(0,estimators_samples.shape[0]):
    a=estimators_samples[i]
    locals()['base'+str(i+1)+'_x_train']=x_train.iloc[a]
    locals()['base'+str(i+1)+'_y_train']=y_train.iloc[a]
    locals()['bag_fit_'+str(i+1)]=locals()['bag_'+str(i+1)].fit(locals()['base'+str(i+1)+'_x_train'], locals()['base'+str(i+1)+'_y_train']) 
    locals()['y_predict_'+str(i+1)]=locals()['bag_'+str(i+1)].predict(x_test)
    bag_predict.append(locals()['y_predict_'+str(i+1)])#存每個Base預測x_test的結果
    
    
#計算ACC
temp=[]#每個模型對某一樣本的預測
acc_count=0
bag_predict=np.array(bag_predict)#list轉array
for i in range(bag_predict.shape[1]):
    temp=[]
    for j in range(bag_predict.shape[0]):
        temp.append(bag_predict[j][i])
    vote= max(temp,key=temp.count)#多數決
    if vote==y_test.iloc[i]:
        acc_count+=1
        
print("acc : ",acc_count/bag_predict.shape[1] )


